# Search for AIMA 4th edition

Implementation of search algorithms and search problems for AIMA.

# Problems and Nodes

We start by defining the abstract class for a `Problem`; specific problem domains will subclass this. To make it easier for algorithms that use a heuristic evaluation function, `Problem` has a default `h` function (uniformly zero), and subclasses can define their own default `h` function.

We also define a `Node` in a search tree, and some functions on nodes: `expand` to generate successors; `path_actions` and `path_states`  to recover aspects of the path from the node.  

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys, os
import time

import numpy as np
import copy

sys.path.append(os.getcwd())
#import bisect
from bisect import *

from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

# Queues

First-in-first-out and Last-in-first-out queues, and a `PriorityQueue`, which allows you to keep a collection of items, and continually remove from it the item with minimum `f(item)` score.

In [3]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# Search Algorithms: Best-First

Best-first search with various *f(n)* functions gives us different search algorithms. Note that A\*, weighted A\* and greedy search can be given a heuristic function, `h`, but if `h` is not supplied they use the problem's default `h` function (if the problem does not define one, it is taken as *h(n)* = 0).

In [4]:

def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    seconds = time.time()
    NODES_EXPANDED = 0
    STATES_REEXPANDED = 0
    FALSE_NODES = 0
    
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    expanded = {}
    while frontier:
        # node extraction, control de nodos DUMMY 
        node = frontier.pop() 
        while (node.path_cost > reached[node.state].path_cost):
            FALSE_NODES += 1
            node = frontier.pop() 
        # Control de estados reexpandidos
        if (node.state in expanded):
            STATES_REEXPANDED +=1
        else:
            expanded[node.state] = True
        # cuenta nodos reexpandidos   
        NODES_EXPANDED += 1
        # Monitorización de nodos expandidos y valores de f()
        #print(node.state, "f()= ", f(node))
       
        #if objetivo, muestra estadisticas
        if problem.is_goal(node.state):
            print("Tiempo de Ejecucion: ", time.time()-seconds)
            print("Nodos Expandidos: ", NODES_EXPANDED)
            print("Estados Alcanzados: ", len(reached))
            print("Nodos en Frontier: ", len(frontier))
            print("Coste de la Solucion: ", node.path_cost)
            print("Longitud de la Solucion: ", len(node))
            print("Nro. de Estados Reexpandidos: ", STATES_REEXPANDED)
            print("Nro. de Nodos False Descartados: ", FALSE_NODES)
            return node
        # node expansion
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
            #if (not s in expanded and (s not in reached or child.path_cost < reached[s].path_cost)): # solo con h() consistente
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



# Other Search Algorithms

Here are the other search algorithms:

In [5]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result != failure:
                return result
        return failure

# Problem Domains

Now we turn our attention to defining some problem domains as subclasses of `Problem`.

# 8 Puzzle Problems

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a goal configuration. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numebred 1 to 8, with 0 representing the blank. An action is the cell index number that is to be swapped with the blank (*not* the actual number to be swapped but the index into the state). So the diagram above left is the state `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, and the action is `8`, because the cell number 8 (the 9th or last cell, the `6` in the bottom right) is swapped with the blank.

There are two disjoint sets of states that cannot be reached from each other. One set has an even number of "inversions"; the other has an odd number. An inversion is when a piece in the state is larger than a piece that follows it.




In [6]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """
    def __init__(self, initial, goal=(1, 2, 3, 8, 0, 4, 7, 6, 5)):
    #def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        #assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can be moved to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    # Costes de acciones distintos de 1
    #def action_cost(self, s, a, s1): return math.exp(s[a]);
    #def action_cost(self, s, a, s1): return s[a]

    
    # heuristic functions
    def h0(self, node):
        """The null heuristic."""
        # es consistente
        return 0

    def h1(self, node):
        """The misplaced tiles heuristic. It has a bug as it counts 0 as a valid tide!!"""
        # es consistente
        return hamming_distance(node.state, self.goal)
    
    def h2_0(self, node):
        # sirve para cualquier estado objetivo
        # es consistente
        suma = 0
        # para cada ficha 1, 2, ..., 8 calcular las distancia de su posicion en node.state a su posicion en self.goal
        # es mas costoso computacionalmente que h2_0
        for i in range(8):
            j=i+1
            p1 = node.state.index(j)
            x1 = p1 % 3
            y1 = p1 // 3
            pg = self.goal.index(j)
            xg = pg % 3
            yg = pg // 3
            suma += abs(x1-xg) + abs(y1-yg)
        return suma
    
    def h2_1(self, node):
        #The Manhattan heuristic, para goal = (1, 2, 3, 8, 0, 4, 7, 6, 5) (tiempo de orden O(n) )
        Xg = (1, 0, 1, 2, 2, 2, 1, 0, 0)
        Yg = (1, 0, 0, 0, 1, 2, 2, 2, 1)
        # Xg[s],Yg[s] = posicion en el objetivo (1, 2, 3, 8, 0, 4, 7, 6, 5) de la ficha s (s = 0, 1, ..., 8)
        return sum(abs(Xg[s] - Xg[g]) + abs(Yg[s] - Yg[g])
                    for (s, g) in zip(node.state, self.goal) if s != 0)
                    # s en state esta en la posicion de g en el goal, i.e., s esta en (Xg[g],Yg[g]) en state
                    # s debe estar en (Xg[s],Yg[s]) en el goal
    
    def h3(self, node):
        # suma 2 por cada ficha a distancia 2 de su posicion en el objetivo
        # es admisible pero no consistente
        # sirve para cualquier estado objetivo
        suma = 0
        # para cada ficha 1, 2, ..., 8 calcular las distancia de su posicion en node.state a su posicion en self.goal
        for i in range(8):
            j=i+1
            p1 = node.state.index(j)
            x1 = p1 % 3
            y1 = p1 // 3
            pg = self.goal.index(j)
            xg = pg % 3
            yg = pg // 3
            if (abs(x1-xg) + abs(y1-yg) == 2):
                suma += 2
        return suma
    
    # Actual heuristic
    def h(self, node): 
       return self.h2_0(node)
      
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different. Corregido para que no considere el 0 como otra ficha"
    return sum(a != b and a!=0 for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')



In [7]:
# Algunos estados alcanzables desde (1, 2, 3, 8, 0, 4, 7, 6, 5)

e05_1 =  EightPuzzle((1, 0, 3, 8, 2, 5, 7, 4, 6))
e10_1 =  EightPuzzle((8, 2, 1, 7, 0, 3, 6, 5, 4))
e15_1 =  EightPuzzle((4, 8, 2, 6, 3, 5, 1, 0, 7))
e20_1 =  EightPuzzle((6, 2, 7, 4, 5, 1, 0, 8, 3))
e21_1 =  EightPuzzle((7, 0, 2, 3, 6, 1, 5, 8, 4))
e23_1 =  EightPuzzle((6, 3, 2, 5, 8, 1, 4, 0, 7))
e24_1 =  EightPuzzle((6, 3, 2, 5, 7, 8, 0, 4, 1))
e24_2 =  EightPuzzle((3, 5, 6, 4, 2, 7, 0, 8, 1))
e25_1 =  EightPuzzle((6, 7, 4, 0, 5, 1, 3, 2, 8))
e30_1 =  EightPuzzle((5, 6, 7, 2, 8, 4, 0, 3, 1))
e30_2 =  EightPuzzle((5, 6, 7, 4, 0, 8, 3, 2, 1))
e30_3 =  EightPuzzle((5, 4, 7, 6, 0, 3, 8, 2, 1))
e30_4 =  EightPuzzle((3, 8, 7, 4, 0, 6, 5, 2, 1))
e30_5 =  EightPuzzle((5, 6, 3, 4, 0, 2, 7, 8, 1))



In [8]:
# Solve an 8 puzzle problem and print out each state
for s in path_states(astar_search(e30_2)):
    print(board8(s))

(5, 6, 7, 4, 0, 8, 3, 2, 1) f()=  24
(5, 0, 7, 4, 6, 8, 3, 2, 1) f()=  24
(5, 6, 7, 4, 8, 0, 3, 2, 1) f()=  24
(5, 6, 7, 4, 2, 8, 3, 0, 1) f()=  24
(5, 6, 7, 0, 4, 8, 3, 2, 1) f()=  24
(5, 7, 0, 4, 6, 8, 3, 2, 1) f()=  24
(5, 6, 7, 4, 2, 8, 0, 3, 1) f()=  24
(0, 6, 7, 5, 4, 8, 3, 2, 1) f()=  24
(5, 6, 7, 3, 4, 8, 0, 2, 1) f()=  24
(5, 6, 7, 4, 2, 8, 3, 1, 0) f()=  24
(5, 6, 0, 4, 8, 7, 3, 2, 1) f()=  24
(5, 6, 7, 4, 8, 1, 3, 2, 0) f()=  24
(0, 5, 7, 4, 6, 8, 3, 2, 1) f()=  24
(5, 7, 8, 4, 6, 0, 3, 2, 1) f()=  26
(6, 0, 7, 5, 4, 8, 3, 2, 1) f()=  26
(5, 6, 7, 4, 8, 1, 3, 0, 2) f()=  26
(4, 5, 7, 0, 6, 8, 3, 2, 1) f()=  26
(5, 6, 7, 0, 2, 8, 4, 3, 1) f()=  26
(5, 6, 7, 3, 4, 8, 2, 0, 1) f()=  26
(5, 0, 6, 4, 8, 7, 3, 2, 1) f()=  26
(5, 6, 7, 4, 2, 0, 3, 1, 8) f()=  26
(5, 7, 8, 4, 6, 1, 3, 2, 0) f()=  26
(5, 6, 7, 4, 8, 1, 0, 3, 2) f()=  26
(0, 5, 6, 4, 8, 7, 3, 2, 1) f()=  26
(5, 6, 0, 4, 2, 7, 3, 1, 8) f()=  26
(4, 5, 7, 3, 6, 8, 0, 2, 1) f()=  26
(0, 6, 7, 5, 2, 8, 4, 3, 1) f()=  26
(

# Symmetric TSP

![](romania.png)

En el TSP simetrico, los estados son pares de la forma [CiudadActual, CiudadesVisitadas]. El estado inicial es [A, ConjuntoVacio], el estado objetivo es de la forma [A, TodasLasCiudades]. Los sucesores de un estado se generan a partir de las ciudades no visitadas, uno por cada una. El coste de una regla es el coste entre la ciudad actual y la nueva ciudad visitada que pasa a ser la actual

In [6]:
class EstadoTSP:
    def __init__(self, actual, visitadas):
        self.actual = actual
        self.visitadas = visitadas
        lista = visitadas.copy()
        lista.append(actual)
        self.state = tuple(lista)
        self.str = " [Visitadas: " + str(self.visitadas) + " Actual: " + str(self.actual) +"]" 
    def __hash__(self):
        return hash(self.state)
    def __eq__(self,other):
        return (self.actual == other.actual) and (self.visitadas == other.visitadas)
    def __repr__(self): 
        return f'{self.str}'  


In [7]:
class Arco:
    def __init__(self,x,y,coste):
        self.x = x
        self.y = y
        self.coste = coste
        self.str = "[("+ str(self.x) + "," + str(self.y) +") " + str(self.coste) + "]"
    def __lt__(self,other):
        return self.coste < other.coste
    def __repr__(self):
        return f'{self.str}'
    def __eq__(self,other):
        return self.x == other.x and self.y == other.y or self.x == other.y and self.y == other.x
        

class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N = int(((8*len(lista)+1)**0.5)-1)/2
        self.Total = 0
        # print(self.N)
        self.ciudades = list(range(int(self.N)))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]
                self.Total += 2*lista[x_acc+y]

    def listaArcosOrdenados(self):
        lista = []
        for x in range(int(self.N)):
            for y in range(x):
                insort(lista,Arco(x,y,self.Dist[x][y]))
        #print(lista)
        return lista
    
    def listaCiudadesGrafoResidual(self,state):
        lista = [0]
        for x in range(int(self.N)):
            if (x not in state.visitadas and x != 0):
                lista.append(x)
        return lista
    
    def listaArcosGrafoResidual(self,state):
        listaCGR = self.listaCiudadesGrafoResidual(state)
        if (len(listaCGR) < 2):
            return []
        if (len(listaCGR) == 2):
            return [Arco(state.actual,0,self.Dist[state.actual][0])]
        lista = []
        for x in listaCGR:
            for y in listaCGR:
                if (x < y and not(x == 0 and y == state.actual)):
                    insort(lista,Arco(x,y,self.Dist[x][y]))
        return lista


    def listaCiudadesAbandonar(self,state):
        lista = []
        for x in range(int(self.N)):
            if (x not in state.visitadas and x != 0):
                lista.append(x)
        return lista
    
    def listaCiudadesAlcanzar(self,state):
        lista = [0]
        for x in range(int(self.N)):
            if (x not in state.visitadas and x != state.actual):
                lista.append(x)
        return lista
    
    def matrizAsignacion(self,state):
        size = int(self.N - len(state.visitadas)) # dimension de la matriz de asignacion
        Asig =  [[0]*size for i in range(size)]
        alcanzar = self.listaCiudadesAlcanzar(state)
        abandonar = self.listaCiudadesAbandonar(state)
        i=0
        for x in range(int(self.N)):
            j=0
            if (x in alcanzar):
                for y in range(int(self.N)):
                    if (y in abandonar):
                        if (x == y or (x == 0 and y == state.actual and size > 1)):
                            Asig[i][j] = self.Total # infinito
                        else:
                            Asig[i][j] = self.Dist[x][y]
                        j += 1
                i += 1
        return Asig



In [8]:
# Implementacion del algoritmo Hungaro tomada de:
# https://transport-systems.imperial.ac.uk/tf/60008_21/n2_5_hungarian_algorithm

def hungarian_step(mat): 
    #The for-loop iterates through every column in the matrix so we subtract this value to every element of the row
    for row_num in range(mat.shape[0]): 
        mat[row_num] = mat[row_num] - np.min(mat[row_num])
    
    #We repeat the process for the columns
    for col_num in range(mat.shape[1]): 
        mat[:,col_num] = mat[:,col_num] - np.min(mat[:,col_num])
    
    return mat

def min_zeros(zero_mat, mark_zero):
    # min_row = [number of zeros, row index number]
    min_row = [99999, -1]

    for row_num in range(zero_mat.shape[0]): 
        if np.sum(zero_mat[row_num] == True) > 0 and min_row[0] > np.sum(zero_mat[row_num] == True):
            min_row = [np.sum(zero_mat[row_num] == True), row_num]

    # Marked the specific row and column as False
    zero_index = np.where(zero_mat[min_row[1]] == True)[0][0]
    mark_zero.append((min_row[1], zero_index))
    zero_mat[min_row[1], :] = False
    zero_mat[:, zero_index] = False

def mark_matrix(mat):
    #Transform the matrix to boolean matrix(0 = True, others = False)
    cur_mat = mat
    zero_bool_mat = (cur_mat == 0)
    zero_bool_mat_copy = zero_bool_mat.copy()

    #Recording possible answer positions by marked_zero
    marked_zero = []
    while (True in zero_bool_mat_copy):
        min_zeros(zero_bool_mat_copy, marked_zero)

    #Recording the row and column indexes seperately.
    marked_zero_row = []
    marked_zero_col = []
    for i in range(len(marked_zero)):
        marked_zero_row.append(marked_zero[i][0])
        marked_zero_col.append(marked_zero[i][1])
    
    # mark rows not containing zeros
    non_marked_row = list(set(range(cur_mat.shape[0])) - set(marked_zero_row))
    
    # mark columns with zeros
    marked_cols = []
    check_switch = True
    while check_switch:
        check_switch = False
        for i in range(len(non_marked_row)):
            row_array = zero_bool_mat[non_marked_row[i], :]
            for j in range(row_array.shape[0]):
                if row_array[j] == True and j not in marked_cols:

                    marked_cols.append(j)
                    check_switch = True

        for row_num, col_num in marked_zero:
            if row_num not in non_marked_row and col_num in marked_cols:
                
                non_marked_row.append(row_num)
                check_switch = True
    
    # mark rows with zeros
    marked_rows = list(set(range(mat.shape[0])) - set(non_marked_row))
    
    return(marked_zero, marked_rows, marked_cols)

def adjust_matrix(mat, cover_rows, cover_cols):
    cur_mat = mat
    non_zero_element = []
    
    # find the minimum value of an element not in a marked column/row 
    for row in range(len(cur_mat)):
        if row not in cover_rows:
            for i in range(len(cur_mat[row])):
                if i not in cover_cols:
                    non_zero_element.append(cur_mat[row][i])
    
    min_num = min(non_zero_element)

    # substract to all values not in a marked row/column
    for row in range(len(cur_mat)):
        if row not in cover_rows:
            for i in range(len(cur_mat[row])):
                if i not in cover_cols:
                    cur_mat[row, i] = cur_mat[row, i] - min_num
    # add to all values in marked rows/column
    for row in range(len(cover_rows)):  
        for col in range(len(cover_cols)):
            cur_mat[cover_rows[row], cover_cols[col]] = cur_mat[cover_rows[row], cover_cols[col]] + min_num

    return cur_mat

def hungarian_algorithm(cost_matrix):
    n = cost_matrix.shape[0]
    cur_mat = copy.deepcopy(cost_matrix)
    
    cur_mat = hungarian_step(cur_mat)
    
    count_zero_lines = 0
        
   
    while count_zero_lines < n:
        ans_pos, marked_rows, marked_cols = mark_matrix(cur_mat)
        count_zero_lines = len(marked_rows) + len(marked_cols)

        if count_zero_lines < n:
            cur_mat = adjust_matrix(cur_mat, marked_rows, marked_cols)
        else:
            return ans_pos
    

In [19]:
class SymmetricTSP(Problem):
    """El problema consiste en encontrar un camino hamiltoniano en un grafo no dirigido
    y totalmente conectado, con costes positivos en los ejes. Las ciudades son 0,...,N-1
    La ciudad de partida es la 0"""

    def __init__(self, grafo):
        self.grafo = grafo
        self.initial = EstadoTSP(0,[])
        #print(self.__dir__initial)
        self.lArcos = grafo.listaArcosOrdenados()
        #print(self.lArcos)

    def actions(self, state):
        """Una accion por cada ciudad no visitada distinta de la actual, si todas visitadas -1"""
        noVisitadas = []
        for c in self.grafo.ciudades:
            if c not in state.visitadas and c != state.actual:
                noVisitadas.append(c)
        if len(noVisitadas)==0:
            noVisitadas = [0]
        return noVisitadas

    def result(self, state, action):
        setVisitadas = list(state.visitadas)
        insort(setVisitadas,state.actual)
        return EstadoTSP(action,setVisitadas)

    def is_goal(self, state):
        return state.actual == 0  and  len(state.visitadas) == self.grafo.N

    def action_cost(self, s, a, s1):
        return self.grafo.Dist[s.actual][a]

   # HEURISTICOS, LO MAS IMPORTANTE!!

    def h_POBRE(self,node):
        # heuristico muy poco informado, cuenta el numero de ciudades que faltan por visitar, es el valor N-k+1
        return self.grafo.N - len(node.state.visitadas)

    def h1(self,node):
        # Heuristico que considera los arcos minimos de las ciudades
        # que quedan por abandonar.
        # No responde a ninguna relajacion, aparentemente
        # Se puede mejorar si consideramos que las ciudades nunca se abandornaran hacia una ciudad ya visitada, excepto la inicial 0
        lista = self.grafo.listaCiudadesAbandonar(node.state)
        h = 0
        for x in lista:
            min = self.lArcos[-1].coste
            for y in range(int(self.grafo.N)):
                if (x!=y and self.grafo.Dist[x][y] < min):
                    min = self.grafo.Dist[x][y]
            h += min
        return h

    def h2(self,node):
        # Relajacion R2, R3, R4.
        # Suma de los N-k+1 arcos de menos coste del grafo residual
        listaAGR = self.grafo.listaArcosGrafoResidual(node.state)
        Nk1 = self.grafo.N - len(node.state.visitadas)
        h = 0
        for i in range(int(Nk1)):
            a = listaAGR.pop(0)
            h += a.coste
        return h

    def h_MST(self, node):
        # Relajacion R3
        # Coste de un arbol de expansion minimo del grafo residual
        # Se calcula con el algoritmo de Kruskal
        if (node.state == self.initial or self.is_goal(node.state)):
            return 0
        listaAGR = self.grafo.listaArcosGrafoResidual(node.state)
        listaCGR = self.grafo.listaCiudadesGrafoResidual(node.state)
        print("node.state", node.state)
        print("actions", self.actions(node.state))
        c = {}
        for x in listaCGR:
            c[x] = x
        nA = self.grafo.N - len(node.state.visitadas)
        h = 0
        while(nA > 0):
            a = listaAGR.pop(0)
            cx = self.particion(c,a.x)
            cy = self.particion(c,a.y)
            #print(cx,cy)
            if (cx != cy):
                h += a.coste
                c[cx] = cy
                nA -= 1
        return h

    def particion(self,c,x):
        if (c[x] == x):
            return x
        else:
            return self.particion(c,c[x])


    def h_HUNGARO(self, node):
        # Relajacion R3
        # Coste de la asignacion minima entre los conjuntos
        # CIUDADES_NO_VISITADAS U {ACTUAL} y CIUDADES_NO_VISITADAS U {INICIAL}
        # Es una aproximacion al coste optimo del problema relajado (R3)
        # ya que pueden aparecer arcos repetidos en la solucion

        if (node.state == self.initial or self.is_goal(node.state)):
            return 0
        #print(node.state)
        cost_matrix = np.asarray(self.grafo.matrizAsignacion(node.state))
        asignacion = hungarian_algorithm(np.asarray(cost_matrix))
        coste = 0
        for asig in asignacion:
            coste += cost_matrix[asig[0],asig[1]]
        return coste

    def h_NN(self, node):        
        # Inicializar la ciudad actual y lista de ciudades por visitar
        actual = node.state.actual
        no_visitadas = [c for c in self.grafo.ciudades if c not in node.state.visitadas]

        h = 0
        
        while no_visitadas:
            # pillar el nearest neighbur
            nearest_city = min(no_visitadas, key=lambda city: self.grafo.Dist[actual][city])
            # añadir al coste
            h += self.grafo.Dist[actual][nearest_city]
            # moverse
            actual = nearest_city
            no_visitadas.remove(nearest_city)

        h += self.grafo.Dist[actual][0]

        return h

    def h_BI(self, node):
        parcial = list(node.state.visitadas)
        if not parcial:
            parcial = [node.state.actual]
        
        no_visitadas = [c for c in self.grafo.ciudades if c not in node.state.visitadas and c != node.state.actual]
        
        h = 0
        
        current_city = node.state.actual
        parcial.append(current_city)
        
        while no_visitadas:
            mejor = float('inf')
            mejor_ciudad = None
            mejor_posicion = None

            for city in no_visitadas:
                for i in range(len(parcial) - 1):
                    increase = (self.grafo.Dist[parcial[i]][city] +
                                self.grafo.Dist[city][parcial[i + 1]] -
                                self.grafo.Dist[parcial[i]][parcial[i + 1]])
                    if increase < mejor:
                        mejor = increase
                        mejor_ciudad = city
                        mejor_posicion = i + 1

            parcial.insert(mejor_posicion, mejor_ciudad)
            h += mejor
            no_visitadas.remove(mejor_ciudad)
        
        h += self.grafo.Dist[parcial[-1]][0]
        
        return h



    # Actual heuristic
    def h(self,node):
        return self.h_BI(node)


In [21]:
# Datos de instancias en formato EDGE_WEIGHT_TYPE de la TSPLIB

ejemploClase = [ 
    0,
    21, 0,
    12, 7, 0,
    15, 32, 5, 0,
    113, 25, 18, 180, 0,
    92, 9, 20, 39, 17, 0
]

gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    # 268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    # 246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    # 121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

instance = SymmetricTSP(Grafo(gr21))
#path_actions(astar_search(instance))
#path_states(astar_search(instance))
path_states(weighted_astar_search(instance, weight=1))


Tiempo de Ejecucion:  82.60293984413147
Nodos Expandidos:  192183
Estados Alcanzados:  385934
Nodos en Frontier:  547489
Coste de la Solucion:  3011
Longitud de la Solucion:  21
Nro. de Estados Reexpandidos:  59131
Nro. de Nodos False Descartados:  28796


[ [Visitadas: [] Actual: 0],
  [Visitadas: [0] Actual: 18],
  [Visitadas: [0, 18] Actual: 6],
  [Visitadas: [0, 6, 18] Actual: 7],
  [Visitadas: [0, 6, 7, 18] Actual: 5],
  [Visitadas: [0, 5, 6, 7, 18] Actual: 15],
  [Visitadas: [0, 5, 6, 7, 15, 18] Actual: 4],
  [Visitadas: [0, 4, 5, 6, 7, 15, 18] Actual: 8],
  [Visitadas: [0, 4, 5, 6, 7, 8, 15, 18] Actual: 2],
  [Visitadas: [0, 2, 4, 5, 6, 7, 8, 15, 18] Actual: 1],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 15, 18] Actual: 14],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 14, 15, 18] Actual: 13],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 13, 14, 15, 18] Actual: 12],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 12, 13, 14, 15, 18] Actual: 17],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 12, 13, 14, 15, 17, 18] Actual: 9],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 17, 18] Actual: 16],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18] Actual: 20],
  [Visitadas: [0, 1, 2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 20] Actu